<div style='float: right'><img src='pic/kanaore.png'/></div>
## <div id='kanaore' />カナオレ

In [ ]:
import numpy as np
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = [s.split(',') for s in """\
1,0,↑,カナオレ
4,0,↑,サムライン
4,1,→,ナンスケ
1,2,↓,ケイスケ
0,3,↓,マツクロ
3,4,↓,フイルオミノ""".split()]
nh, nw = 5, 5
def makecand(lst, y, x, d, l, p, u):
    yy, xx = y+[0, -1, 0, 1][d], x+[-1, 0, 1, 0][d]
    if 0 <= yy < nh and 0 <= xx < nw and (yy,xx) not in u:
        if p == l - 1:
            lst.append(u + [(yy, xx)])
            return
        for k in range(4):
            makecand(lst, yy, xx, k, l, p + 1, u + [(yy,xx)])

### 問題
* 全マスに１文字ずつ入れ、リストの言葉全部を盤面に作ります
* どの言葉も、１文字目は言葉の前に書かれている数字のマスに入り、２文字目は矢印の方向に１つ進んだマスに入ります
* ３文字目以降は、タテヨコにつながって入ります
* １つの文字を複数の言葉が共通して使うこともあります
* １つの文字を１つの言葉が複数回使うことはありません

### 変数
* v：各マスの文字のID (1)
* 各単語ごとに候補(lst)を作り
  * vt：どの候補を選ぶか (2)

### 制約
* 候補から選ぶこと (3)
* 候補を選んだら、その場所は、その文字とすること (4)

In [ ]:
m = LpProblem()
v = addvars(nh, nw) # (1)
dic, dic2 = {}, {}
for wd in data:
    for c in wd[3]:
        if not c in dic:
            t = len(dic)
            dic[c], dic2[t] = t, c
    lst = []
    y, x = int(wd[0]), int(wd[1])
    makecand(lst, y,x, '←↑→↓'.index(wd[2]), len(wd[3]), 1, [(y,x)])
    vt = addbinvars(len(lst)) # (2)
    m += lpSum(vt) == 1 # (3)
    for i, cand in enumerate(lst):
        for j, (y,x) in enumerate(cand):
            c = dic[wd[3][j]]
            m += v[y][x] <= c + nw*nh*(1-vt[i]) # (4)
            m += v[y][x] >= c - nw*nh*(1-vt[i]) # (4)
%time m.solve()
print('\n'.join(' '.join(s) for s in np.vectorize(lambda x: dic2[int(value(x))])(v)))